[View in Colaboratory](https://colab.research.google.com/github/jsedoc/Sharvin-Shah-Spring-2018/blob/master/tree2seq_colab.ipynb)

In [21]:
import sys
print(sys.version_info)

!pip install http://download.pytorch.org/whl/cu90/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl
!pip install tqdm nltk

print(torch.cuda.device_count())

sys.version_info(major=3, minor=6, micro=3, releaselevel='final', serial=0)
1


In [46]:
!rm -r data synth_t2t_data.zip
!wget http://www.seas.upenn.edu/~joao/synth_t2t_data.zip
!unzip synth_t2t_data.zip
!pwd

--2018-05-01 20:06:39--  http://www.seas.upenn.edu/~joao/synth_t2t_data.zip
Resolving www.seas.upenn.edu (www.seas.upenn.edu)... 158.130.68.91, 2607:f470:8:64:5ea5::9
Connecting to www.seas.upenn.edu (www.seas.upenn.edu)|158.130.68.91|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10835 (11K) [application/zip]
Saving to: ‘synth_t2t_data.zip’

synth_t2t_data.zip  100%[===================>]  10.58K  --.-KB/s    in 0s      

2018-05-01 20:06:39 (114 MB/s) - ‘synth_t2t_data.zip’ saved [10835/10835]

Archive:  synth_t2t_data.zip
   creating: data/
  inflating: data/train.orig         
/content


In [0]:
from tqdm import tqdm
from copy import deepcopy

PAD = 0
UNK = 1
BOS = 2
EOS = 3

PAD_WORD = '<blank>'
UNK_WORD = '<unk>'
BOS_WORD = '<s>'
EOS_WORD = '</s>'

MAX_LENGTH = 512


import torch
import torch.utils.data as data

from nltk import Tree

datapath = '/content/data/train.orig'

class Dataset(data.Dataset):

    def __len__(self):
        return self.size

    def read_trees(self, filename):
        with open(filename, 'r') as f:
            trees = [Tree.fromstring(line) for line in tqdm(f.readlines())]
        return trees
    
    def read_seqs(self, filename):
        with open(filename, 'r') as f:
            seqs = [line for line in tqdm(f.readlines())]
        return seqs

In [48]:
dataset = Dataset()
trees = dataset.read_trees('/content/data/train.orig')
print(trees[1])

100%|██████████| 1000/1000 [00:00<00:00, 33761.86it/s]

(* (* (+ 55 (+ 56 53)) 31) (- 5 (* 54 9)))


In [0]:
def create_vocab(filename):
    vocab = dict()
    with open(filename, 'r') as f:
        for token in f.read().split():
            if token not in vocab:
                vocab[token] = 1
            else:
                vocab[token] += 1
    print(len(vocab.keys()))    
    index = 0
    token_dict = {}            
    for token in vocab.keys():
        token_dict[token] = index
        index += 1
    
    return token_dict

def fetch_unique_index(token_dict, token):
    return token_dict[token]

def create_one_hot(token_dict):
    one_hot_dict = {}
    vector_dim = len(token_dict.keys())
    for token in token_dict:    
        tensor = torch.zeros(1, vector_dim)
        tensor[0][token_dict[token]] = 1
        one_hot_dict[token] = tensor
    
    return one_hot_dict


def fetch_one_hot(one_hot_dict, token):
    return one_hot_dict[token]

In [50]:
vocab = create_vocab(datapath)
print(vocab)
oh_vocab = create_one_hot(vocab)
print(fetch_one_hot(oh_vocab, '4'))

71
{'(': 0, '+': 1, '43': 2, '47': 3, ')': 4, '*': 5, '55': 6, '56': 7, '53': 8, '31': 9, '-': 10, '5': 11, '54': 12, '9': 13, '/': 14, '4': 15, '1': 16, '39': 17, '50': 18, '23': 19, '52': 20, '51': 21, '30': 22, '57': 23, '42': 24, '11': 25, '6': 26, '20': 27, '59': 28, '0': 29, '58': 30, '19': 31, '29': 32, '38': 33, '28': 34, '27': 35, '16': 36, '44': 37, '60': 38, '34': 39, '36': 40, '32': 41, '35': 42, '8': 43, '33': 44, '7': 45, '12': 46, '3': 47, '15': 48, '21': 49, '13': 50, '17': 51, '18': 52, '61': 53, '49': 54, '10': 55, '40': 56, '2': 57, '46': 58, '48': 59, '14': 60, '37': 61, '45': 62, '22': 63, '26': 64, '41': 65, '24': 66, '25': 67, '62': 68, '63': 69, '64': 70}


Columns 0 to 12 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 13 to 25 
    0     0     1     0     0     0     0     0     0     0     0     0     0

Columns 26 to 38 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 39 to 51 
  

In [51]:
dataset = Dataset()
trees = dataset.read_trees(datapath)
seqs = dataset.read_seqs(datapath)

print(trees[1])
print(seqs[1])

100%|██████████| 1000/1000 [00:00<00:00, 1264868.52it/s]

(* (* (+ 55 (+ 56 53)) 31) (- 5 (* 54 9)))
( * ( * ( + 55 ( + 56 53 ) ) 31 ) ( - 5 ( * 54 9 ) ) )



In [0]:
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable as Var

PAD = 0
UNK = 1
BOS = 2
EOS = 3

PAD_WORD = '<blank>'
UNK_WORD = '<unk>'
BOS_WORD = '<s>'
EOS_WORD = '</s>'

# module for childsumtreelstm
class ChildSumTreeLSTM(nn.Module):
    def __init__(self, in_dim, mem_dim):
        super(ChildSumTreeLSTM, self).__init__()
        self.in_dim = in_dim
        self.mem_dim = mem_dim
        self.ioux = nn.Linear(self.in_dim, 3 * self.mem_dim)
        self.iouh = nn.Linear(self.mem_dim, 3 * self.mem_dim)
        self.fx = nn.Linear(self.in_dim, self.mem_dim)
        self.fh = nn.Linear(self.mem_dim, self.mem_dim)

    def node_forward(self, inputs, child_c, child_h):
        child_h_sum = torch.sum(child_h, dim=0, keepdim=True)
        print("dimension of inputs: ", inputs.size())
        print("dimension of child_h_sum: ", child_h_sum.size())
        iou = self.ioux(inputs) + self.iouh(child_h_sum)
        i, o, u = torch.split(iou, iou.size(1) // 3, dim=1)
        i, o, u = F.sigmoid(i), F.sigmoid(o), F.tanh(u)

        f = F.sigmoid(
            self.fh(child_h) +
            self.fx(inputs).repeat(len(child_h), 1)
        )
        fc = torch.mul(f, child_c)

        c = torch.mul(i, u) + torch.sum(fc, dim=0, keepdim=True)
        h = torch.mul(o, F.tanh(c))
        return c, h

    def forward(self, tree):
        print(type(tree))
        print(tree)
        if type(tree) == Tree:
            child_states = []
            for child in tree:
                child_states.append(self.forward(child))
            child_c, child_h = zip(* map(lambda x: x, child_states))
            child_c, child_h = torch.cat(child_c, dim=0), torch.cat(child_h, dim=0)
            
            if type(tree) == str:
                node_label_repr = Var(fetch_one_hot(oh_vocab, tree))
            else:
                node_label_repr = Var(fetch_one_hot(oh_vocab, tree.label()))
            state = self.node_forward(node_label_repr, child_c, child_h)
        else:
            if type(tree) == str:
                node_label_repr = Var(fetch_one_hot(oh_vocab, tree))
            child_c = Var(node_label_repr.data.new(1, self.mem_dim).fill_(0.))
            child_h = Var(node_label_repr.data.new(1, self.mem_dim).fill_(0.))
            state = self.node_forward(node_label_repr, child_c, child_h)
        # Using tree.label() instead of torch.zeros() ensures that the device placement is inherited  
      
        return state
    


# module for distance-angle similarity
class Similarity(nn.Module):
    def __init__(self, mem_dim, hidden_dim, num_classes):
        super(Similarity, self).__init__()
        self.mem_dim = mem_dim
        self.hidden_dim = hidden_dim
        self.num_classes = num_classes
        self.wh = nn.Linear(2 * self.mem_dim, self.hidden_dim)
        self.wp = nn.Linear(self.hidden_dim, self.num_classes)

    def forward(self, lvec, rvec):
        mult_dist = torch.mul(lvec, rvec)
        abs_dist = torch.abs(torch.add(lvec, -rvec))
        vec_dist = torch.cat((mult_dist, abs_dist), 1)

        out = F.sigmoid(self.wh(vec_dist))
        out = F.log_softmax(self.wp(out), dim=1)
        return out


# putting the whole model together
class SimilarityTreeLSTM(nn.Module):
    def __init__(self, vocab_size, in_dim, mem_dim, hidden_dim, num_classes, sparsity, freeze):
        super(SimilarityTreeLSTM, self).__init__()
        self.emb = nn.Embedding(vocab_size, in_dim, padding_idx=PAD, sparse=sparsity)
        if freeze:
            self.emb.weight.requires_grad = False
        self.childsumtreelstm = ChildSumTreeLSTM(in_dim, mem_dim)
        self.similarity = Similarity(mem_dim, hidden_dim, num_classes)

    def forward(self, ltree, linputs, rtree, rinputs):
        linputs = self.emb(linputs)
        rinputs = self.emb(rinputs)
        lstate, lhidden = self.childsumtreelstm(ltree, linputs)
        rstate, rhidden = self.childsumtreelstm(rtree, rinputs)
        output = self.similarity(lstate, rstate)
        return output

In [53]:
# encoder input size (in_dim) could be anything
# encoder hidden size (mem_dim) would be equal to the dimension of the vector
# encoder hidden size could be anything, decoder hidden size is dependent on final encoder hidden state size
# decoder output size could be anything
cs_treelstm = ChildSumTreeLSTM(71, 2)
print(cs_treelstm)
out = cs_treelstm.forward(trees[1])

ChildSumTreeLSTM(
  (ioux): Linear(in_features=71, out_features=6)
  (iouh): Linear(in_features=2, out_features=6)
  (fx): Linear(in_features=71, out_features=2)
  (fh): Linear(in_features=2, out_features=2)
)
<class 'nltk.tree.Tree'>
(* (* (+ 55 (+ 56 53)) 31) (- 5 (* 54 9)))
<class 'nltk.tree.Tree'>
(* (+ 55 (+ 56 53)) 31)
<class 'nltk.tree.Tree'>
(+ 55 (+ 56 53))
<class 'str'>
55
dimension of inputs:  torch.Size([1, 71])
dimension of child_h_sum:  torch.Size([1, 2])
<class 'nltk.tree.Tree'>
(+ 56 53)
<class 'str'>
56
dimension of inputs:  torch.Size([1, 71])
dimension of child_h_sum:  torch.Size([1, 2])
<class 'str'>
53
dimension of inputs:  torch.Size([1, 71])
dimension of child_h_sum:  torch.Size([1, 2])
dimension of inputs:  torch.Size([1, 71])
dimension of child_h_sum:  torch.Size([1, 2])
dimension of inputs:  torch.Size([1, 71])
dimension of child_h_sum:  torch.Size([1, 2])
<class 'str'>
31
dimension of inputs:  torch.Size([1, 71])
dimension of child_h_sum:  torch.Size([1, 2])


In [0]:
from copy import deepcopy

In [0]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax()

    def forward(self, input, hidden):
        print("input size: ", input.size())
        print("hidden size: ", hidden.size())
        output = self.embedding(input).view(1, 1, -1)
        print("After Embedding, size: ", output.size())
        output = F.relu(output)
        print("output size after ReLU: ", output.size())
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result

In [0]:
def token_to_index(token_dict, filename):
    file_embedding = []
    
    with open(filename, 'r') as f:
        for line in f.readlines()[:2]:
            line_embedding = []
            print("line: ", line)
            tokens = line.split()
            print("tokens: ", tokens)
            for token in tokens:
                line_embedding.append(token_dict[token])
            
            print("line embedding: ", line_embedding)
            line_tensor = Var(torch.LongTensor(line_embedding).view(-1, 1))
            print("line tensor: ", line_tensor)
            file_embedding.append(line_tensor)
            print("file embedding so far: ", file_embedding)
    
    return file_embedding

In [57]:
emb = token_to_index(vocab, datapath)
emb[1]

line:  ( + 43 47 )

tokens:  ['(', '+', '43', '47', ')']
line embedding:  [0, 1, 2, 3, 4]
line tensor:  Variable containing:
 0
 1
 2
 3
 4
[torch.LongTensor of size 5x1]

file embedding so far:  [Variable containing:
 0
 1
 2
 3
 4
[torch.LongTensor of size 5x1]
]
line:  ( * ( * ( + 55 ( + 56 53 ) ) 31 ) ( - 5 ( * 54 9 ) ) )

tokens:  ['(', '*', '(', '*', '(', '+', '55', '(', '+', '56', '53', ')', ')', '31', ')', '(', '-', '5', '(', '*', '54', '9', ')', ')', ')']
line embedding:  [0, 5, 0, 5, 0, 1, 6, 0, 1, 7, 8, 4, 4, 9, 4, 0, 10, 11, 0, 5, 12, 13, 4, 4, 4]
line tensor:  Variable containing:
    0
    5
    0
    5
    0
    1
    6
    0
    1
    7
    8
    4
    4
    9
    4
    0
   10
   11
    0
    5
   12
   13
    4
    4
    4
[torch.LongTensor of size 25x1]

file embedding so far:  [Variable containing:
 0
 1
 2
 3
 4
[torch.LongTensor of size 5x1]
, Variable containing:
    0
    5
    0
    5
    0
    1
    6
    0
    1
    7
    8
    4
    4
    9
    4
    0
   10

Variable containing:
    0
    5
    0
    5
    0
    1
    6
    0
    1
    7
    8
    4
    4
    9
    4
    0
   10
   11
    0
    5
   12
   13
    4
    4
    4
[torch.LongTensor of size 25x1]

In [58]:
out[1].size()

torch.Size([1, 2])

In [59]:
len(vocab.keys())

71

In [60]:
decoder = DecoderRNN(out[1].size()[1], len(vocab.keys()))
#hidden = decoder.initHidden()
encoder_outputs = out[1]
encoder_outputs = encoder_outputs.unsqueeze(0)
print(encoder_outputs)
for i in range(out[1].size()[1]):
  out, hidden = decoder.forward(emb[1][i], encoder_outputs)

Variable containing:
(0 ,.,.) = 
  0.2348 -0.3339
[torch.FloatTensor of size 1x1x2]

input size:  torch.Size([1])
hidden size:  torch.Size([1, 1, 2])
After Embedding, size:  torch.Size([1, 1, 2])
output size after ReLU:  torch.Size([1, 1, 2])
input size:  torch.Size([1])
hidden size:  torch.Size([1, 1, 2])
After Embedding, size:  torch.Size([1, 1, 2])
output size after ReLU:  torch.Size([1, 1, 2])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [61]:
out

Variable containing:

Columns 0 to 9 
-5.0534 -4.6585 -4.1347 -4.4704 -3.9207 -5.1462 -4.0097 -3.6419 -4.4979 -3.7046

Columns 10 to 19 
-4.6966 -4.4008 -4.3315 -4.0058 -4.5834 -4.0512 -3.6190 -4.0552 -4.6586 -4.7106

Columns 20 to 29 
-3.9462 -3.5929 -3.8332 -4.6592 -4.7064 -3.9936 -4.1536 -3.9050 -4.1274 -4.2782

Columns 30 to 39 
-4.4027 -4.7186 -3.9313 -4.6256 -3.3866 -4.5186 -4.9716 -4.3361 -5.2079 -3.8708

Columns 40 to 49 
-4.4456 -3.3501 -4.5868 -4.8579 -4.4372 -4.1422 -4.6728 -5.0819 -4.3704 -4.8397

Columns 50 to 59 
-4.1965 -4.6392 -4.1611 -4.1451 -4.4054 -4.7596 -3.9315 -3.8555 -4.7831 -3.7334

Columns 60 to 69 
-5.0541 -5.1496 -4.1554 -4.5686 -4.8764 -4.4532 -5.1694 -4.5786 -4.2481 -4.8181

Columns 70 to 70 
-3.8813
[torch.FloatTensor of size 1x71]

In [62]:
def train(tree_variable, target_variable, 
          encoder, decoder, encoder_optimizer, decoder_optimizer, 
          criterion, max_length=MAX_LENGTH):
  encoder_optimizer.zero_grad()
  decoder_optimizer.zero_grad()
  
  # encode tree
  
  # decoder serial and accumulate loss
  
  # call backward
  
  # update parameters

NameError: ignored